## time score

In [1]:
import pandas as pd
from datetime import datetime

In [7]:
deposits = pd.read_csv('deposits.csv')
withdraws = pd.read_csv('withdraws.csv')
swaps = pd.read_csv('swaps.csv')

deposits['datetime'] = pd.to_datetime(deposits['timestamp'], unit='s')
withdraws['datetime'] = pd.to_datetime(withdraws['timestamp'], unit='s')
swaps['datetime'] = pd.to_datetime(swaps['timestamp'], unit='s')

all_events = pd.concat([
    deposits[['account_id', 'datetime']],
    withdraws[['account_id', 'datetime']],
    swaps[['account_id', 'datetime']]
])

In [5]:
print(" deposits.csv columns:")
print(deposits.columns.tolist())

print("\n withdraws.csv columns:")
print(withdraws.columns.tolist())

print("\nswaps.csv columns:")
print(swaps.columns.tolist())

 deposits.csv columns:
['account_id', 'amountUSD', 'hash', 'inputTokenAmounts', 'inputTokens', 'pool_id', 'pool_name', 'tickLower', 'tickUpper', 'timestamp', 'datetime']

 withdraws.csv columns:
['account_id', 'amountUSD', 'hash', 'inputTokenAmounts', 'inputTokens', 'pool_id', 'pool_name', 'reserveAmounts', 'tickLower', 'tickUpper', 'timestamp', 'datetime']

swaps.csv columns:
['account_id', 'amountIn', 'amountInUSD', 'amountOut', 'amountOutUSD', 'hash', 'pool_id', 'timestamp', 'tokenIn_id', 'tokenIn_symbol', 'tokenOut_id', 'tokenOut_symbol']


In [9]:
first_tx = all_events.groupby('account_id')['datetime'].min().reset_index()

# Extract date, time, and number of days since first transaction
first_tx['date'] = first_tx['datetime'].dt.date
first_tx['time'] = first_tx['datetime'].dt.time
first_tx['number_of_days_in_total'] = (datetime.utcnow() - first_tx['datetime']).dt.days


In [17]:
def compute_time_score(days):
    score = 20  # base score
    if days >= 7:
        score += 10
    if days >= 30:
        score += 15
    if days >= 90:
        score += 15
    if days >= 180:
        score += 20
    if days >= 365:
        score += 10
    if days > 365:
        score += 10
    return min(score, 100)

first_tx['time_score'] = first_tx['number_of_days_in_total'].apply(compute_time_score)


time_df = first_tx[['account_id', 'date', 'time', 'number_of_days_in_total', 'time_score']]

In [19]:
print(time_df.head())

                                   account_id        date      time  \
0  0x000000000057e8abd581828a2b40076852c0ba6e  2021-05-21  13:33:50   
1  0x000000000eaaa544c48dbfc6d752cd4ad6ba8a56  2021-05-29  21:22:50   
2  0x0000000049b1c7e404f93c4ed429e2cd1f990aa7  2021-06-22  14:46:22   
3  0x000000007cb2bd00ae5eb839930bb7847ae5b039  2021-06-09  09:17:15   
4  0x00000000d70742d790f9936f25d414dbce6818b0  2025-04-05  06:22:23   

   number_of_days_in_total  time_score  
0                     1461         100  
1                     1452         100  
2                     1429         100  
3                     1442         100  
4                       46          45  


In [21]:
time_df.shape

(11408, 5)

## deposits

In [23]:
deposits['datetime'] = pd.to_datetime(deposits['timestamp'], unit='s').dt.round('1s')
withdraws['datetime'] = pd.to_datetime(withdraws['timestamp'], unit='s').dt.round('1s')

# --- Step 1: Identify Bot Deposits ---
withdraw_info = withdraws[['account_id', 'datetime', 'amountUSD']].copy()
withdraw_info.rename(columns={'datetime': 'datetime_withdraw', 'amountUSD': 'amountUSD_withdraw'}, inplace=True)

merged = pd.merge(
    deposits[['account_id', 'datetime', 'amountUSD']],
    withdraw_info,
    on='account_id',
    how='left'
)

merged['time_diff'] = (merged['datetime_withdraw'] - merged['datetime']).dt.total_seconds()
merged['value_diff'] = abs(merged['amountUSD_withdraw'] - merged['amountUSD']) / merged['amountUSD']
merged['is_bot_flag'] = (merged['time_diff'] >= 0) & (merged['time_diff'] <= 60) & (merged['value_diff'] <= 0.02)

bot_flags = (
    merged.groupby(['account_id', 'datetime'])['is_bot_flag']
    .max()
    .reset_index()
    .rename(columns={'is_bot_flag': 'is_bot'})
)

deposits = pd.merge(deposits, bot_flags, on=['account_id', 'datetime'], how='left')
deposits['is_bot'] = deposits['is_bot'].fillna(False)

# --- Step 2: Adjusted Total Deposit Volume ---
deposits['effective_amountUSD'] = deposits.apply(
    lambda row: row['amountUSD'] * 0.1 if row['is_bot'] else row['amountUSD'],
    axis=1
)

total_deposit = deposits.groupby('account_id')['effective_amountUSD'].sum().reset_index()
total_deposit.columns = ['account_id', 'total_deposit_all_time']

def volume_score(amount):
    if amount >= 100000:
        return 250
    elif amount >= 50000:
        return 200
    elif amount >= 10000:
        return 150
    elif amount >= 1000:
        return 100
    elif amount >= 100:
        return 50
    else:
        return 20

total_deposit['volume_score'] = total_deposit['total_deposit_all_time'].apply(volume_score)

# --- Step 3: Frequency Score ---
deposits['day'] = deposits['datetime'].dt.date
deposits['day_weight'] = deposits['is_bot'].apply(lambda x: 0.2 if x else 1.0)
deposits['month'] = deposits['datetime'].dt.to_period('M')

daily_weights = deposits.groupby(['account_id', 'day'])['day_weight'].max().reset_index()

user_activity_range = deposits.groupby('account_id')['datetime'].agg(['min', 'max']).reset_index()
user_activity_range['months_active'] = ((user_activity_range['max'] - user_activity_range['min']) / pd.Timedelta(days=30)).clip(lower=1)

weighted_days = daily_weights.groupby('account_id')['day_weight'].sum().reset_index()
weighted_days.columns = ['account_id', 'total_weighted_days']

freq = pd.merge(weighted_days, user_activity_range[['account_id', 'months_active']], on='account_id')
freq['avg_monthly_deposit_days'] = freq['total_weighted_days'] / freq['months_active']

def frequency_score(avg):
    if avg >= 8:
        return 125
    elif avg >= 4:
        return 100
    elif avg >= 2:
        return 75
    elif avg >= 1:
        return 50
    else:
        return 25

freq['frequency_score'] = freq['avg_monthly_deposit_days'].apply(frequency_score)

# --- Step 4: Bot Score ---
bot_counts = deposits.groupby('account_id')['is_bot'].sum().reset_index()
bot_counts.columns = ['account_id', 'num_bot_deposits']

def compute_bot_score(n):
    if n == 0:
        return 125
    elif n <= 3:
        return 112.5
    elif n <= 10:
        return 100
    elif n <= 50:
        return 50
    else:
        return 0

bot_counts['bot_score'] = bot_counts['num_bot_deposits'].apply(compute_bot_score)
bot_counts['bot_transaction'] = bot_counts['num_bot_deposits'] > 0

# --- Step 5: Merge All Subscores ---
deposit_df = total_deposit.merge(freq[['account_id', 'avg_monthly_deposit_days', 'frequency_score']], on='account_id', how='outer')
deposit_df = deposit_df.merge(bot_counts, on='account_id', how='outer')

deposit_df['deposit_score'] = deposit_df['volume_score'] + deposit_df['frequency_score'] + deposit_df['bot_score']

# Fill NaNs for accounts with only swaps
deposit_df = deposit_df.fillna({
    'total_deposit_all_time': 0,
    'volume_score': 0,
    'avg_monthly_deposit_days': 0,
    'frequency_score': 0,
    'num_bot_deposits': 0,
    'bot_score': 125,
    'bot_transaction': False,
    'deposit_score': 0
})

# --- Step 6: Merge with time_df ---
time_df = pd.merge(time_df, deposit_df, on='account_id', how='left')

# Fill any remaining NaNs
time_df = time_df.fillna({
    'total_deposit_all_time': 0,
    'volume_score': 0,
    'avg_monthly_deposit_days': 0,
    'frequency_score': 0,
    'num_bot_deposits': 0,
    'bot_score': 125,
    'bot_transaction': False,
    'deposit_score': 0
})

C:\Users\dhana\AppData\Local\Temp\ipykernel_80368\2444935644.py:125: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  time_df = time_df.fillna({


In [25]:
print(time_df.head())

                                   account_id        date      time  \
0  0x000000000057e8abd581828a2b40076852c0ba6e  2021-05-21  13:33:50   
1  0x000000000eaaa544c48dbfc6d752cd4ad6ba8a56  2021-05-29  21:22:50   
2  0x0000000049b1c7e404f93c4ed429e2cd1f990aa7  2021-06-22  14:46:22   
3  0x000000007cb2bd00ae5eb839930bb7847ae5b039  2021-06-09  09:17:15   
4  0x00000000d70742d790f9936f25d414dbce6818b0  2025-04-05  06:22:23   

   number_of_days_in_total  time_score  total_deposit_all_time  volume_score  \
0                     1461         100                0.000000           0.0   
1                     1452         100                0.000000           0.0   
2                     1429         100                0.000000           0.0   
3                     1442         100                0.000000           0.0   
4                       46          45              476.239385          50.0   

   avg_monthly_deposit_days  frequency_score  num_bot_deposits  bot_score  \
0              

## withdraw

In [33]:
import pandas as pd
from datetime import datetime

# Load deposits and withdraws
deposits = pd.read_csv("deposits.csv")
withdraws = pd.read_csv("withdraws.csv")

# Convert timestamps
deposits['datetime'] = pd.to_datetime(deposits['timestamp'], unit='s')
withdraws['datetime'] = pd.to_datetime(withdraws['timestamp'], unit='s')

# Sort both by account_id and datetime
deposits.sort_values(by=['account_id', 'datetime'], inplace=True)
withdraws.sort_values(by=['account_id', 'datetime'], inplace=True)

# Dictionary to collect holding times
holding_times = {}

# Grouped access for speed
grouped_deposits = deposits.groupby('account_id')
grouped_withdraws = withdraws.groupby('account_id')

# Iterate through accounts that have both deposits and withdraws
common_accounts = set(deposits['account_id']) & set(withdraws['account_id'])

for acc in common_accounts:
    d_times = grouped_deposits.get_group(acc)['datetime'].tolist()
    w_times = grouped_withdraws.get_group(acc)['datetime'].tolist()

    d_idx = 0
    local_holds = []

    for w in w_times:
        # Move forward through deposit list to find the last deposit before this withdrawal
        while d_idx < len(d_times) and d_times[d_idx] <= w:
            d_idx += 1
        if d_idx == 0:
            continue  # No deposit before this withdrawal
        hold = (w - d_times[d_idx - 1]).total_seconds() / (60 * 60 * 24)
        local_holds.append(hold)

    if local_holds:
        holding_times[acc] = sum(local_holds) / len(local_holds)

# Convert to DataFrame
holding_df = pd.DataFrame.from_dict(holding_times, orient='index', columns=['avg_holding_days'])
holding_df.reset_index(inplace=True)
holding_df.rename(columns={'index': 'account_id'}, inplace=True)

# Holding time score
def holding_time_score(days):
    if days >= 90: return 120
    elif days >= 30: return 90
    elif days >= 7: return 60
    elif days >= 1: return 30
    elif days >= (1 / 24): return 10
    else: return 1

holding_df['holding_time_score'] = holding_df['avg_holding_days'].apply(holding_time_score)

# Now merge with withdraw frequency table (same as before)
withdraw_counts = pd.DataFrame()
withdraw_counts['num_deposits'] = deposits.groupby('account_id').size()
withdraw_counts['num_withdrawals'] = withdraws.groupby('account_id').size()
withdraw_counts = withdraw_counts.fillna(0).reset_index()
withdraw_counts.rename(columns={'index': 'account_id'}, inplace=True)

# Withdrawal frequency ratio
withdraw_counts['withdraw_freq_ratio'] = withdraw_counts['num_withdrawals'] / withdraw_counts['num_deposits'].replace(0, 1)

def withdrawal_frequency_score(ratio):
    if ratio <= 0.1: return 80
    elif ratio <= 0.3: return 60
    elif ratio <= 0.6: return 40
    elif ratio <= 1.0: return 20
    else: return 5

withdraw_counts['withdraw_freq_score'] = withdraw_counts['withdraw_freq_ratio'].apply(withdrawal_frequency_score)

# Merge holding and frequency scores
withdraw_df = pd.merge(withdraw_counts, holding_df, on='account_id', how='left')
withdraw_df['avg_holding_days'] = withdraw_df['avg_holding_days'].fillna(0)
withdraw_df['holding_time_score'] = withdraw_df['holding_time_score'].fillna(120)  # reward long-term holders

# Final withdraw score (Max 200)
withdraw_df['withdraw_score'] = withdraw_df['withdraw_freq_score'] + withdraw_df['holding_time_score']

# Merge with time_df
time_df = pd.merge(time_df, withdraw_df, on='account_id', how='left')
time_df = time_df.fillna({
    'num_deposits': 0,
    'num_withdrawals': 0,
    'withdraw_freq_ratio': 0,
    'withdraw_freq_score': 80,
    'avg_holding_days': 0,
    'holding_time_score': 120,
    'withdraw_score': 200
})


In [37]:
time_df.head()

,account_id,date,time,number_of_days_in_total,time_score,total_deposit_all_time,volume_score,avg_monthly_deposit_days,frequency_score,num_bot_deposits,bot_score,bot_transaction,deposit_score,num_deposits,num_withdrawals,withdraw_freq_ratio,withdraw_freq_score,avg_holding_days,holding_time_score,withdraw_score
0,0x000000000057e8abd581828a2b40076852c0ba6e,2021-05-21,13:33:50,1461,100,0.000000,0.0,0.0,0.0,0.0,125.0,False,0.0,0.0,0.0,0.0,80.0,0.0,120.0,200.0
1,0x000000000eaaa544c48dbfc6d752cd4ad6ba8a56,2021-05-29,21:22:50,1452,100,0.000000,0.0,0.0,0.0,0.0,125.0,False,0.0,0.0,0.0,0.0,80.0,0.0,120.0,200.0
2,0x0000000049b1c7e404f93c4ed429e2cd1f990aa7,2021-06-22,14:46:22,1429,100,0.000000,0.0,0.0,0.0,0.0,125.0,False,0.0,0.0,0.0,0.0,80.0,0.0,120.0,200.0
3,0x000000007cb2bd00ae5eb839930bb7847ae5b039,2021-06-09,09:17:15,1442,100,0.000000,0.0,0.0,0.0,0.0,125.0,False,0.0,0.0,0.0,0.0,80.0,0.0,120.0,200.0
4,0x00000000d70742d790f9936f25d414dbce6818b0,2025-04-05,06:22:23,46,45,476.239385,50.0,1.0,50.0,0.0,125.0,False,225.0,1.0,0.0,0.0,80.0,0.0,120.0,200.0


## swap

In [53]:
import pandas as pd

# Load swap data and process it
swaps = pd.read_csv("swaps.csv")
swaps['datetime'] = pd.to_datetime(swaps['timestamp'], unit='s')
swaps['month'] = swaps['datetime'].dt.to_period('M')
swaps['fee_usd'] = swaps['amountOutUSD'] - swaps['amountInUSD']
swaps['fee_usd'] = swaps['fee_usd'].apply(lambda x: x if x > 0 else 0)

# Total fee per account
fee_by_account = swaps.groupby('account_id')['fee_usd'].sum().reset_index()
fee_by_account.rename(columns={'fee_usd': 'total_fee_generated_usd'}, inplace=True)

def fee_volume_score(fee):
    if fee >= 1_000_000:
        return 120
    elif fee >= 500_000:
        return 100
    elif fee >= 100_000:
        return 80
    elif fee >= 10_000:
        return 40
    elif fee > 0:
        return 20
    else:
        return 0

fee_by_account['fee_volume_score'] = fee_by_account['total_fee_generated_usd'].apply(fee_volume_score)

# Monthly activity per account
monthly_activity = swaps.groupby(['account_id', 'month']).size().reset_index(name='tx_count')
monthly_counts = monthly_activity.groupby('account_id').size().reset_index(name='active_swap_months')

def activity_score(months):
    if months >= 12:
        return 80
    elif months >= 6:
        return 60
    elif months >= 3:
        return 40
    elif months >= 1:
        return 20
    else:
        return 0

monthly_counts['swap_activity_score'] = monthly_counts['active_swap_months'].apply(activity_score)

# Final swap_df
swap_df = pd.merge(fee_by_account, monthly_counts, on='account_id', how='outer').fillna(0)
swap_df['swap_score'] = swap_df['fee_volume_score'] + swap_df['swap_activity_score']
swap_df = swap_df[[
    'account_id',
    'total_fee_generated_usd',
    'fee_volume_score',
    'active_swap_months',
    'swap_activity_score',
    'swap_score'
]]

# Merge swap score into time_df
final_df = pd.merge(time_df, swap_df, on='account_id', how='left')
final_df = final_df.fillna({
    'total_fee_generated_usd': 0,
    'fee_volume_score': 0,
    'active_swap_months': 0,
    'swap_activity_score': 0,
    'swap_score': 0
})

# Add total_score column
final_df['total_score'] = (
    final_df['time_score'] +
    final_df['deposit_score'] +
    final_df['withdraw_score'] +
    final_df['swap_score']
)
final_df = final_df.sort_values(by='total_score', ascending=False)
# Save final result
final_df.to_csv("final_account_scores.csv", index=False)
